In [1]:
from shared.utils.import_utils import import_or_reload
from dotenv import load_dotenv
load_dotenv('./.env_staging')

True

In [2]:
import os
from sqlalchemy.engine import URL

# Database connection parameters
db_url = URL.create(
    drivername="postgresql",
    username=os.getenv("DB_USERNAME"),
    password=None,
    host=os.getenv("DB_HOST"),
    port=5432,
    database=os.getenv("DB_DATABASE"),
)

os.environ["SQL_DATABASE_URL"] = db_url.render_as_string(hide_password=True)

from shared.db.session import engine

session.py: successful connexion to PostgreSQL !
session.py: database URL: postgresql://cedric@easygy-aurora-cluster.cluster-cywqf0ulc6ar.eu-west-3.rds.amazonaws.com:5432/easy_crm_uat


In [4]:
from sqlalchemy import MetaData

# --- 1. Connexion à Postgres via SQLAlchemy
metadata = MetaData()

# Réflexion du schéma « public »
metadata.reflect(bind=engine, schema="public")

In [20]:
from neo4j import GraphDatabase
from sqlalchemy.dialects.postgresql import ENUM

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
NEO4J_URI = os.getenv("NEO4J_URI", "")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "")

AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)

with neo4j_driver.session() as sess:

    sess.run("MATCH (n) DETACH DELETE n")

    # a) Créer un nœud Class par table
    for table in metadata.tables.values():
        sess.run(
            "MERGE (c:Class {_entity_name: $table})",
            table=table.name
        )

    # b) Pour chaque table, ajouter ses colonnes comme propriétés
    for table in metadata.tables.values():
        # Crée un dictionnaire des propriétés pour cette table
        # { 'colonne1': 'type1', 'colonne2': 'type2', ... }
        properties = {col.name: str(col.type) for col in table.columns}
        
        # Met à jour le nœud existant en lui ajoutant toutes les propriétés
        sess.run(
            """
            MATCH (c:Class {_entity_name: $table_name})
            SET c += $props
            """,
            table_name=table.name,
            props=properties
        )

    # c) Créer les relations d’après les ForeignKeys SQLAlchemy
    for table in metadata.tables.values():
        for fk in table.foreign_keys:
            src = table.name
            tgt = fk.column.table.name
            col   = fk.parent.name
            sess.run(
                """
                MATCH (c1:Class {n_entity_name: $src})
                MATCH (c2:Class {_entity_name: $tgt})
                MERGE (c1)-[r:RELATES_TO {via: $col}]->(c2)
                """,
                src=src, tgt=tgt, col=col
            )

    # d) Créer les nœuds et relations pour les types ENUM
    for table in metadata.tables.values():
        for col in table.columns:
            if isinstance(col.type, ENUM):
                enum_name = col.type.name
                enum_values = col.type.enums
                
                sess.run(
                    """
                    // Trouve la table parente
                    MATCH (c:Class {_entity_name: $table_name})
                    
                    // Crée (ou trouve) le nœud pour le type ENUM
                    MERGE (e:Enum {_entity_name: $enum_name})
                    
                    // Définit ses valeurs possibles
                    SET e.values = $enum_values
                    
                    // Crée la relation entre la table et le type ENUM
                    MERGE (c)-[r:HAS_ENUM {column: $column_name}]->(e)
                    """,
                    table_name=table.name,
                    column_name=col.name,
                    enum_name=enum_name,
                    enum_values=enum_values
                )


"""
MATCH p = (c1:Class)-[:RELATES_TO]->()
WHERE c1.name IN ["lead", "account", "contact", "product", "product_data", "opportunity", "contract", "quote_specification", "quote", "quote_option" , "town", "network_system_operator", "delivery_point", "delivery_point_data", "reading", "delivery_point_customer_mandate", "link_contract_delivery_point_data"] 
RETURN p
"""

'\nMATCH p = (c1:Class)-[:RELATES_TO]->()\nWHERE c1.name IN ["lead", "account", "contact", "product", "product_data", "opportunity", "contract", "quote_specification", "quote", "quote_option" , "town", "network_system_operator", "delivery_point", "delivery_point_data", "reading", "delivery_point_customer_mandate", "link_contract_delivery_point_data"] \nRETURN p\n'

In [27]:
import openai
import json
from sqlalchemy import create_engine, MetaData
from sqlalchemy.sql.sqltypes import Enum as SQLEnum
from pydantic import create_model
from typing import Optional
import inspect
import os

# CONFIGURATION
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY
CACHE_FILE = "model_field_cache.json"

# INITIALISATION
metadata = MetaData()
metadata.reflect(bind=engine)

# Charger cache si existant
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE) as f:
        cache = json.load(f)
else:
    cache = {}

def describe_columns(table):
    lines = []
    for col in table.columns:
        dtype = str(col.type)
        if isinstance(col.type, SQLEnum):
            enum_vals = ", ".join(col.type.enums)
            dtype += f" (Enum: {enum_vals})"
        lines.append(f"- {col.name}: {dtype}")
    return "\n".join(lines)


def ask_llm_to_filter(columns_description, table_name):
    if table_name in cache:
        return cache[table_name]

    prompt = f"""
    Voici les colonnes de la table `{table_name}` :
    {columns_description}

    Garde uniquement les colonnes qui représentent vraisemblablement un concept pouvant faire partie d'une ontologie décrivant un business et ignore les colonnes techniques (ID internes, timestamps, etc.).

    Réponds uniquement avec une liste JSON des noms de colonnes à garder.
    """
    from openai import OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    content = response.choices[0].message.content
    print("--- LLM response content ---")
    print(repr(content))
    result = json.loads(content)
    cache[table_name] = result

    with open(CACHE_FILE, "w") as f:
        json.dump(cache, f, indent=2)

    return result
    
def generate_model_code(table, kept_fields):
    lines = [f"class {table.name.capitalize()}(BaseModel):"]
    for col in table.columns:
        if col.name in kept_fields:
            # Détection très simplifiée des types
            typename = "Optional[str]"
            if isinstance(col.type, SQLEnum):
                typename = f"Optional[Literal[{', '.join(repr(v) for v in col.type.enums)}]]"
            lines.append(f"    {col.name}: {typename} = None")
    return "\n".join(lines)

# 🏗 Générer tous les modèles
print("from typing import Optional, Literal")
print("from pydantic import BaseModel\n")

for table in metadata.tables.values():
    col_desc = describe_columns(table)
    kept_fields = ask_llm_to_filter(col_desc, table.name)
    code = generate_model_code(table, kept_fields)
    print(code, "\n")

from typing import Optional, Literal
from pydantic import BaseModel

--- LLM response content ---
'```json\n[\n    "master_company",\n    "first_name",\n    "last_name",\n    "email",\n    "is_active",\n    "role",\n    "status",\n    "origin",\n    "phone_number",\n    "mobile_phone_number",\n    "avatar_icon_url"\n]\n```'


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [25]:
response

NameError: name 'response' is not defined

In [ ]:
import openai
import json
from sqlalchemy import create_engine, MetaData
from sqlalchemy.sql.sqltypes import Enum as SQLEnum
from pydantic import create_model
from typing import Optional
import inspect
import os

# 🔧 CONFIGURATION
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY
CACHE_FILE = "model_field_cache.json"

# 🧠 INITIALISATION
metadata = MetaData()
metadata.reflect(bind=engine)

# ⏳ Charger cache si existant
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE) as f:
        cache = json.load(f)
else:
    cache = {}

def describe_columns(table):
    lines = []
    for col in table.columns:
        dtype = str(col.type)
        if isinstance(col.type, SQLEnum):
            enum_vals = ", ".join(col.type.enums)
            dtype += f" (Enum: {enum_vals})"
        lines.append(f"- {col.name}: {dtype}")
    return "\n".join(lines)

def ask_llm_to_filter(columns_description, table_name):
    if table_name in cache:
        return cache[table_name]

    prompt = f"""
Voici les colonnes de la table `{table_name}` :
{columns_description}

Garde uniquement les colonnes qui peuvent être comprises ou mentionnées par un humain dans un email, un compte-rendu ou une transcription.
Ignore les colonnes techniques (ID internes, timestamps, etc.).

Réponds uniquement avec une liste JSON des noms de colonnes à garder.
"""
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    result = json.loads(response.choices[0].message.content)
    cache[table_name] = result

    with open(CACHE_FILE, "w") as f:
        json.dump(cache, f, indent=2)

    return result

def generate_model_code(table, kept_fields):
    lines = [f"class {table.name.capitalize()}(BaseModel):"]
    for col in table.columns:
        if col.name in kept_fields:
            # Détection très simplifiée des types
            typename = "Optional[str]"
            if isinstance(col.type, SQLEnum):
                typename = f"Optional[Literal[{', '.join(repr(v) for v in col.type.enums)}]]"
            lines.append(f"    {col.name}: {typename} = None")
    return "\n".join(lines)

# 🏗 Générer tous les modèles
print("from typing import Optional, Literal")
print("from pydantic import BaseModel\n")

for table in metadata.tables.values():
    col_desc = describe_columns(table)
    kept_fields = ask_llm_to_filter(col_desc, table.name)
    code = generate_model_code(table, kept_fields)
    print(code, "\n")
